In [1]:
import pandas as pd
from pandas import Series,DataFrame

In [2]:
# read datafiles
dfapart = pd.read_csv("../data/curated/apartment_facilities.csv")
dfhouse = pd.read_csv('../data/curated/house_facilities.csv')
income = pd.read_csv('../data/curated/income_person.csv')

In [3]:
population = pd.read_csv("../data/raw/population_migration.csv")
population = population.rename(columns={'sa2_name_2016':'suburb'})
income = income.rename(columns={'SA2 NAME':'suburb', '2014-15':'2014income', \
                                '2015-16':'2015income', '2016-17':'2016income', \
                                '2017-18':'2017income', '2018-19':'2018income'})
income['suburb'] = income['suburb'].str.upper()
population['suburb'] = population['suburb'].str.upper()

In [7]:
# separate years
pop16 = population.loc[:,['suburb', 'erp_2016', 'natural_increase_2016_17', \
                       'net_internal_migration_2016_17', 'net_overseas_migration_2016_17']]
pop17 = population.loc[:,['suburb', 'erp_2017', 'natural_increase_2017_18', \
                       'net_internal_migration_2017_18', 'net_overseas_migration_2017_18']]
pop18 = population.loc[:,['suburb', 'erp_2018', 'natural_increase_2018_19', \
                       'net_internal_migration_2018_19', 'net_overseas_migration_2018_19']]

In [6]:
# find the suburb of each address
dfapart["suburb"] = dfapart["address"].str.extract("(.+?)\d")
dfapart["suburb"] = dfapart["suburb"].str.strip().str.upper()

dfhouse["suburb"] = dfhouse["address"].str.extract("(.+?)\d")
dfhouse["suburb"] = dfhouse["suburb"].str.strip().str.upper()

apa = dfapart.loc[:,["price", "suburb", "beds", "baths", \
                 "parking", "latitudes", "longitudes", \
                 "school", "train_station", "fitness_gym", \
                 "hospital", "shopping_center", "park"]]
house = dfhouse.loc[:,["price", "suburb", "beds", "baths", \
                 "parking", "latitudes", "longitudes", \
                 "school", "train_station", "fitness_gym", \
                 "hospital", "shopping_center", "park"]]

In [8]:
house["suburb"] = house["suburb"].apply(str)
apa["suburb"] = apa["suburb"].apply(str)

In [9]:
# list SA2 names
sub = population["suburb"]
sub = sub.dropna(how ="any")
sub = sub.str.upper().to_list()

In [ ]:
# match the corresponding area of house
from difflib import get_close_matches
for i in range(len(house)):
    area = house["suburb"][i]
    s = get_close_matches(area, sub, n=1)
    house["suburb"][i] = s
    if len(house["suburb"][i])==0:
        house["suburb"][i] = ''
    else:
        house["suburb"][i] = house["suburb"][i][0]    
        

In [ ]:
# match the corresponding area of apartment
for i in range(len(apa)):
    area = apa["suburb"][i]
    s = get_close_matches(area, sub, n=1)
    apa["suburb"][i] = s
    if len(apa["suburb"][i])==0:
        apa["suburb"][i] = ''
    else:
        apa["suburb"][i] = apa["suburb"][i][0]   
        

In [12]:
# separate years
income16 = income.loc[:,["suburb", "2016income"]]
income17 = income.loc[:,["suburb", "2017income"]]
income18 = income.loc[:,["suburb", "2018income"]]

In [13]:
# match domain and population and income
matchapart1 = pd.merge(apa,pop16, on = ['suburb'])
matchapart2 = pd.merge(matchapart1,income16, on = ['suburb'])
len(matchapart2)

836

In [14]:
matchhouse1 = pd.merge(dfhouse,pop16, on = ['suburb'])
matchhouse2 = pd.merge(matchhouse1,income16, on = ['suburb'])
len(matchhouse2)

487

In [25]:
# keep useful features
features = ["price", "suburb", "beds", "baths", \
                 "parking", "latitudes", "longitudes", \
                 "school", "train_station", "fitness_gym", \
                 "hospital", "shopping_center", "park", 'erp_2016', 'natural_increase_2016_17',
       'net_internal_migration_2016_17', 'net_overseas_migration_2016_17', '2016income']

In [29]:
matchhouse2 = matchhouse2.loc[:,features]

In [28]:
matchapart2 = matchapart2.loc[:,features]

In [30]:
matchapart2.to_csv('../data/curated/match_apartment16.csv')
matchhouse2.to_csv('../data/curated/match_house16.csv')

In [37]:
pop_income = pd.merge(population, income, on = ['suburb'])

In [42]:
list_features = ['suburb', 'erp_2016', 'erp_2017', 'erp_2018', 'natural_increase_2016_17', \
                 'net_internal_migration_2016_17','net_overseas_migration_2016_17', \
                 'natural_increase_2017_18', 'net_internal_migration_2017_18', 'net_overseas_migration_2017_18',\
       'natural_increase_2018_19', 'net_internal_migration_2018_19', 'net_overseas_migration_2018_19', \
                 '2016income', '2017income', '2018income']
pop_income = pop_income.loc[:, list_features]

In [46]:
pop_income.to_csv("../data/curated/population & income from 2016-2018")